In [1]:
import pandas as pd 
from utils import *

In [2]:
RUN_GPT_EVAL = False

# Filterting

In [3]:
fn = '../Data/raw_data.xlsx'
df = pd.read_excel(fn, skiprows=[1])

/Users/danielhendriks/miniforge3/envs/llms-in-medicine/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
# Ignore test data, study launched at 21-11-2024
df = df[df['StartDate'] > pd.to_datetime('2024-11-21 00:00:00')]
df.shape

(159, 599)

In [5]:
# consent provided, 1 = Yes
df = df[df.Q3 == 1]

In [6]:
# include participants that finished the study
print('Samples before dropping:', df.shape[0])
c = df.Finished == 0
print('Samples that did not finish: ', (c).sum())

df = df[~c]
print('Samples after dropping:', df.shape[0])

Samples before dropping: 156
Samples that did not finish:  42
Samples after dropping: 114


In [7]:
# assert that all included participants have a condition assigned to them
print('Samples before dropping:', df.shape[0])
c = pd.isna(df.condition)
print('Samples without condition: ', (c).sum())

df = df[~c]
print('Samples after dropping:', df.shape[0])

Samples before dropping: 114
Samples without condition:  0
Samples after dropping: 114


In [8]:
# exclude participants that failed the attention check
# for the control group, no attention check was displayed, hence the value is Nan
attention_check = pd.isna(df['Q19_7']) | (df['Q19_7'] == 5)

print('Samples before dropping:', df.shape[0])
print('Samples failing attention check: ', (~attention_check).sum())

df = df[attention_check]
print('Samples after dropping:', df.shape[0])

Samples before dropping: 114
Samples failing attention check:  11
Samples after dropping: 103


In [9]:
# Exclude duplicates
print("Duplicates:", df.ui.duplicated().sum())
df = df[~df.ui.duplicated()]

Duplicates: 2


In [10]:
# Filtered dataset includes 101 participants
df.shape

(101, 599)

In [11]:
cols_con = [col for col in df.columns if col.startswith('q_control')]
cols_std = [col for col in df.columns if col.startswith('q_standard')]
cols_cot = [col for col in df.columns if col.startswith('q_cot')]
cols_dif = [col for col in df.columns if col.startswith('q_dd')]

# Prepare dataframe

In [12]:
# turn df into long form 
df_long = pd.melt(df, id_vars=['ResponseId', 'condition'], 
        value_vars=cols_con+cols_std+cols_cot+cols_dif,
        var_name='question', value_name='answer') \
    .dropna(axis=0, subset='answer')
    
# preprocessing: strip and lower 
df_long['answer'] = df_long.answer.str.strip().str.lower()

In [13]:
# extract question_n to merge df's
def map_question(q):
    if 'control' in q: return f"question_{q.split('control')[-1]}"
    elif 'standard' in q: return f'question_{q.split("standard")[-1]}'
    elif 'cot' in q: return f'question_{q.split("cot")[-1]}'
    elif 'dd' in q: return f'question_{q.split("dd")[-1]}'
    else: raise ValueError
    
df_long['question'] = df_long.question.map(map_question)
df_long.head(3)

,ResponseId,condition,question,answer
1,R_7Bl1xOlCKLlDuUk,1.0,question_1,intussesseption
5,R_3L27f1b3klEk0fL,1.0,question_1,intussusception
6,R_6jkMqDRXLPUtGyd,1.0,question_1,intussusseption


# Map judegment from GPT-4 and radiologists

In [14]:
fn = '../data/judgement_rad.xlsx'
df_map_rad = pd.read_excel(fn,usecols=['question','answer','judgement radiologist','ground_truth'])
df_map_rad = df_map_rad.rename({'judgement radiologist': 'judgement'},axis=1)
df_map_rad['answer'] = df_map_rad['answer'].astype(str)
df_map_rad['judgement'] = df_map_rad.judgement.str.capitalize()


In [15]:
df_map_rad.judgement.value_counts(dropna=False)

judgement
Yes        423
No         336
Partial     53
Name: count, dtype: int64

In [16]:
# merge df in long format with evals from radiologists (rad) and then with evals from gpt
# finally, merge with additional data from df such as times and other constructs' data
df_eval = pd.merge(left=df_long, right=df_map_rad,on=['question', 'answer'], 
                       how='left',validate='many_to_one')


In [17]:
df_eval.judgement.value_counts(dropna=False)

judgement
Yes        1341
No          546
Partial     133
Name: count, dtype: int64

In [18]:

# df_eval = pd.merge(left=df_eval_rad, right=df_map_gpt,on=['question', 'answer'], 
#                    how='left',suffixes=('_rad','_gpt'),validate='many_to_one')

df_eval = pd.merge(left=df_eval.drop('condition',axis=1), right=df, 
                   how='left',on=['ResponseId'],validate='many_to_one')

df_eval.head(3)

,ResponseId,question,answer,ground_truth,judgement,StartDate,EndDate,Status,IPAddress,Progress,...,t_dd12,t_dd13,t_dd14,t_dd15,t_dd16,t_dd17,t_dd18,t_dd19,t_dd20,condition
0,R_7Bl1xOlCKLlDuUk,question_1,intussesseption,Colocolonic intussusception,Partial,2024-11-21 12:45:03,2024-11-21 13:49:31,0,104.28.55.217,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,R_3L27f1b3klEk0fL,question_1,intussusception,Colocolonic intussusception,Partial,2024-11-21 15:37:29,2024-11-21 16:00:10,0,108.20.119.209,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,R_6jkMqDRXLPUtGyd,question_1,intussusseption,Colocolonic intussusception,Partial,2024-11-21 15:58:22,2024-11-21 16:26:07,0,12.48.162.126,100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [19]:
# preprocess columns with values for constructs like cognitive load
cols = ['Q17_1','Q17_2','Q17_3','Q17_4','Q17_5']
df_eval['cognitive-load'] = (df_eval[cols].sum(axis=1,skipna=False) - 5) / (35 - 5) * 100

cols = ['Q19_1', 'Q19_2', 'Q19_3', 'Q19_4', 'Q19_5', 'Q19_6']
df_eval['usefulness'] = (df_eval[cols].sum(axis=1,skipna=False) - 6) / (42 - 6) * 100

cols = ['Q809_1', 'Q809_2', 'Q809_3', 'Q809_4', 'Q809_5', 'Q809_6']
df_eval['ease-of-use'] = (df_eval[cols].sum(axis=1,skipna=False) - 6) / (42 - 6) * 100

cols = ['Q21_1','Q21_2','Q21_3','Q21_4','Q21_5','Q21_6']
df_eval['trust'] = (df_eval[cols].sum(axis=1,skipna=False) - 6) / (42 - 6) * 100

In [20]:
df_eval.shape

(2020, 607)

In [21]:
df_eval.to_excel('../Data/data_evaluated.xlsx', index=False)

In [22]:
df_eval

,ResponseId,question,answer,ground_truth,judgement,StartDate,EndDate,Status,IPAddress,Progress,...,t_dd16,t_dd17,t_dd18,t_dd19,t_dd20,condition,cognitive-load,usefulness,ease-of-use,trust
0,R_7Bl1xOlCKLlDuUk,question_1,intussesseption,Colocolonic intussusception,Partial,2024-11-21 12:45:03,2024-11-21 13:49:31,0,104.28.55.217,100,...,0.0,0.0,0.0,0.0,0.0,1.0,40.000000,NaN,NaN,NaN
1,R_3L27f1b3klEk0fL,question_1,intussusception,Colocolonic intussusception,Partial,2024-11-21 15:37:29,2024-11-21 16:00:10,0,108.20.119.209,100,...,0.0,0.0,0.0,0.0,0.0,1.0,46.666667,NaN,NaN,NaN
2,R_6jkMqDRXLPUtGyd,question_1,intussusseption,Colocolonic intussusception,Partial,2024-11-21 15:58:22,2024-11-21 16:26:07,0,12.48.162.126,100,...,0.0,0.0,0.0,0.0,0.0,1.0,60.000000,NaN,NaN,NaN
3,R_3o5eW5zH6mVOqJt,question_1,intussuscpetion,Colocolonic intussusception,Partial,2024-11-21 16:37:23,2024-11-21 17:01:03,0,162.17.72.102,100,...,0.0,0.0,0.0,0.0,0.0,1.0,16.666667,NaN,NaN,NaN
4,R_3lBxjyrPfIoDrBD,question_1,intussusception,Colocolonic intussusception,Partial,2024-11-21 21:47:59,2024-11-21 22:06:10,0,146.75.222.0,100,...,0.0,0.0,0.0,0.0,0.0,1.0,20.000000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015,R_747WuS5rSwsR7Xj,question_20,neurocysticercosis,Neurocysticercosis,Yes,2024-11-28 20:03:49,2024-11-28 20:29:12,0,184.99.22.107,100,...,16440.0,14016.0,14856.0,93275.0,15705.0,4.0,50.000000,80.555556,83.333333,47.222222
2016,R_72EBK9amk6ack3p,question_20,neurocysticercosis,Neurocysticercosis,Yes,2024-11-28 22:32:55,2024-11-28 22:45:22,0,73.71.168.243,100,...,21775.0,9902.0,6152.0,22784.0,17982.0,4.0,56.666667,83.333333,72.222222,61.111111
2017,R_1hRvBHltOuR13y4,question_20,neurocysticercosis,Neurocysticercosis,Yes,2024-11-28 22:37:49,2024-11-28 22:58:58,0,69.116.137.206,100,...,41785.0,62164.0,55346.0,38689.0,53077.0,4.0,26.666667,75.000000,63.888889,52.777778
2018,R_59EUTcxWBa4nq2B,question_20,neurocysticercosis,Neurocysticercosis,Yes,2024-11-29 17:42:08,2024-11-29 18:18:26,0,174.201.16.183,100,...,11165.0,27321.0,15145.0,59954.0,21927.0,4.0,43.333333,66.666667,63.888889,50.000000
